In [3]:
import random
from math import sin

def initialization(population_size, chromosome_length):
    population = []
    for _ in range(population_size):
        chromosome = [random.choice([0, 1]) for _ in range(chromosome_length)]
        population.append(chromosome)
    return population

# Convert binary to integer
def convert(binary_string):
    x = int(binary_string, 2)
    return sin(x)

def tournament_selection(population):
    # 1. Generate a random number. [0, 1)
    random_number_i = random.random()

    # 2. Multiply total the size of the population (all 𝑥s present), which is len(population) here,
    #    with this random number.
    i = int(random_number_i * len(population))

    # 3. Keep the integer part of the result, call it 𝑖, and take out 𝑥𝑖 from the population.
    selected_individual_i = population[i]

    # 4. Find 𝑦𝑖 = sin(𝑥𝑖)
    fitness_i = convert(''.join(map(str, selected_individual_i)))

    # 5. Generate a random number.
    random_number_j = random.random()

    # 6. Multiply total the size of the population (all 𝑥s present), which is len(population) here,
    #    with this random number.
    j = int(random_number_j * len(population))

    # 7. Keep the integer part of the result, call it 𝑗.
    selected_individual_j = population[j]

    # 8. Find 𝑦𝑗 = sin 𝑥𝑗
    fitness_j = convert(''.join(map(str, selected_individual_j)))

    # 9. Find max {𝑦𝑖, 𝑦𝑗}
    max_fitness = max(fitness_i, fitness_j)

    return selected_individual_i if max_fitness == fitness_i else selected_individual_j

def crossover(parent1, parent2):
    # Generate a random number and multiply by the chromosome length
    random_number = int(random.random() * len(parent1))

    # Create children using crossover
    child1 = parent1[:random_number] + parent2[random_number:]
    child2 = parent2[:random_number] + parent1[random_number:]

    return child1, child2

def mutate_chromosome(chromosome):
    # Generate a random number and multiply it with the chromosome length
    random_number_bit = int(random.random() * len(chromosome))

    # Complement the bit in the selected chromosome
    mutated_chromosome = chromosome.copy()
    mutated_chromosome[random_number_bit] = 1 - mutated_chromosome[random_number_bit]

    return mutated_chromosome

def repeat_process(population_size, chromosome_length, generations):
    population = initialization(population_size, chromosome_length)
    for generation in range(generations):
        # Selection
        parent1 = tournament_selection(population)
        parent2 = tournament_selection(population)

        # Crossover
        child1, child2 = crossover(parent1, parent2)

        # Mutation
        child1 = mutate_chromosome(child1)
        child2 = mutate_chromosome(child2)

        # Replace parents with children
        population.remove(parent1)
        population.remove(parent2)
        population.extend([child1, child2])

    # Return the final population
    return population


ValueError: either both or neither of x and y should be given

In [7]:
import numpy as np

# Generate a number between 0 to 1
def generate_x(population):
    return np.random.rand(population)

# Selection
def selection(x, population):
    c = np.round(x * population, 2)
    return np.sin(np.radians(c))

# Sort the y maximum to minimum
def sorting(y):
    return np.sort(y)[::-1]

# Rejected 20% population
def crossover(y, population):
    selected_population_no = int(population - population * 20 / 100)
    sel_pop = y[:selected_population_no]

    # Generate random numbers
    r = np.random.rand(selected_population_no // 2)
    p1 = np.random.choice(sel_pop, selected_population_no // 2)
    p2 = np.random.choice(sel_pop, selected_population_no // 2)

    mask = (r <= 0.5)
    b = np.where(mask, (2 * r) ** (1 / 21), (2 * (1 - r)) ** (1 / 21))

    # One more condition to write
    mask = (r > 0)
    b = np.where(mask, (1 / 2 * (1 - r)) ** (1 / 21), 0)

    ch1 = 0.5 * ((1 + b) * p1 + (1 - b) * p2)
    ch2 = 0.5 * ((1 - b) * p1 + (1 + b) * p2)

    ch = np.concatenate([ch1, ch2])
    return np.where(ch >= 0, 1, 0)

# Polynomial Mutation
def polynomial_mutation(parent, mutation_operator):
    r = np.random.rand(len(parent))
    mask = (r <= 0.5)
    d = np.where(mask, (2 * r) ** (1 / (mutation_operator + 1)) - 1, 1 - (2 * (1 - r)) ** (1 / (mutation_operator + 1)))
    mutated_child = parent + d
    return mutated_child

# Example usage
population_size = 50
x = generate_x(population_size)

for generation in range(500):
    y = selection(x, population_size)
    sorted_y = sorting(y)
    children = crossover(sorted_y, population_size)

    # Mutation
    mutated_children = polynomial_mutation(children, mutation_operator=20)

    # Evaluate fitness
    fitness_values = [np.max(y), np.max(sorted_y), np.max(children), np.max(mutated_children)]

    # Print results at the end of 500 generations
    if generation == 499:
        max_fitness_index = np.argmax(fitness_values)
        max_fitness_chromosome = mutated_children if max_fitness_index == 3 else np.eye(10)[max_fitness_index]
        max_fitness_binary = ''.join(map(str, max_fitness_chromosome.astype(int)))
        max_fitness_decimal = int(max_fitness_binary, 2)
        max_fitness_degrees = max_fitness_decimal * 360 / (2**10 - 1)

        # Print the maximum fitness chromosome and its corresponding angle
        print("Maximum Fitness Chromosome:", max_fitness_chromosome)
        print("Maximum Fitness (Decimal):", max_fitness_decimal)
        print("Maximum Fitness (Degrees):", max_fitness_degrees)

        # Print the maximum fitness value
        max_fitness_value = np.max(fitness_values)
        print("Maximum Fitness Value:", max_fitness_value)


Maximum Fitness Chromosome: [1.08616243 1.01859912 1.08544521 0.99911904 0.8302344  1.01450683
 0.98989614 1.02326889 1.05219244 1.0058493  0.98485898 1.0502583
 1.02641991 0.95998864 1.00731842 1.01201678 0.96732803 1.00103354
 0.80947947 0.97918994 0.87199081 1.10336519 1.01680958 1.00903955
 0.99407899 0.98165659 1.01566993 1.04903889 0.88549777 1.01475002
 1.05773158 0.9492765  1.00728965 1.03187234 0.99822318 1.02912003
 1.03367143 1.00036258 1.0280707  1.0536743 ]
Maximum Fitness (Decimal): 987226388191
Maximum Fitness (Degrees): 347411045697.7126
Maximum Fitness Value: 1.1033651853629909
